In [4]:
%pip install tabulate

import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Read the Excel file
df = pd.read_excel('C:\\Users\\Yourisha\\Documents\\SEM 6\\florai\\ai-model\\data\\gbif_cleaned.xlsx')

# Display the first 5 rows
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

# Print the column names and their data types
print(df.info())

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


| gbifID     | datasetKey                           | occurrenceID                                  | kingdom   | phylum       | class         | order     | family     | genus   | species           | infraspecificEpithet   | taxonRank   | scientificName          | verbatimScientificName   | verbatimScientificNameAuthorship   | countryCode   | locality                                                                                                                                                                      | stateProvince   | occurrenceStatus   | individualCount   | publishingOrgKey                     | decimalLatitude   | decimalLongitude   | coordinateUncertaintyInMeters   | coordinatePrecision   | elevation   | elevationAccuracy   | depth   | depthAccuracy   | eventDate        | day   | month   | year   | taxonKey   | speciesKey   | basisOfRecord      | institutionCode   | collectionCode   | catalogNumber   | recordNumber   | identifiedBy   | dateIdentified   | license   | r

In [5]:
# Select relevant columns
columns_to_keep = ['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']
df_cleaned = df[columns_to_keep]

# Drop rows with missing values in specified columns
df_cleaned = df_cleaned.dropna(subset=['decimalLatitude', 'decimalLongitude', 'eventDate'])

# Save to CSV
df_cleaned.to_csv('cleaned_gbif_data.csv', index=False)

# Print info
print(f"Original shape: {df.shape}")
print(f"Cleaned shape: {df_cleaned.shape}")
print(df_cleaned.head().to_markdown(index=False, numalign="left", stralign="left"))

Original shape: (729, 50)
Cleaned shape: (671, 4)
| species           | decimalLatitude   | decimalLongitude   | eventDate           |
|:------------------|:------------------|:-------------------|:--------------------|
| Mikania micrantha | 1.2               | 110.3              | 2004-11-29T00:00    |
| Mikania micrantha | 19.1755           | 99.0336            | 2019-12-18T00:00    |
| Mikania micrantha | 14.6254           | 121.124            | 2024-12-13T15:15:19 |
| Mikania micrantha | 3.72269           | 101.75             | 2025-10-19T23:22    |
| Mikania micrantha | 14.5587           | 121.136            | 2025-01-09T12:31:37 |


In [ ]:
import pandas as pd
import requests
import time

# 1. Load your cleaned dataset
df = pd.read_csv('cleaned_gbif_data.csv') 


# Ensure eventDate is datetime objects to extract the date part easily
df['eventDate'] = pd.to_datetime(df['eventDate'], format='mixed', errors='coerce')

def get_historical_weather(lat, lon, date):
    """
    Fetches Max Temp and Precipitation Sum for a specific location and date
    from Open-Meteo Archive API.
    """
    date_str = date.strftime('%Y-%m-%d')
    url = "https://archive-api.open-meteo.com/v1/archive"
    
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": date_str,
        "end_date": date_str,
        "daily": ["temperature_2m_max", "precipitation_sum"],
        "timezone": "auto"
    }
    
    try:
        response = requests.get(url, params=params)
        data = response.json()
        
        if 'daily' in data:
            temp_max = data['daily']['temperature_2m_max'][0]
            precip_sum = data['daily']['precipitation_sum'][0]
            return temp_max, precip_sum
    except Exception as e:
        print(f"Error fetching data for {lat}, {lon}: {e}")
        
    return None, None

# 2. Iterate and Fetch Data
print("Fetching weather data... this may take a moment.")

# Create lists to store new data
temps = []
precips = []

for index, row in df.iterrows():
    lat = row['decimalLatitude']
    lon = row['decimalLongitude']
    date = row['eventDate']
    
    # Check if date is valid (not null)
    if pd.notnull(date):
        temp, precip = get_historical_weather(lat, lon, date)
        temps.append(temp)
        precips.append(precip)
        
        # Be polite to the API! Sleep for a fraction of a second
        time.sleep(0.2)
    else:
        temps.append(None)
        precips.append(None)

# 3. Add to DataFrame
df['max_temp_c'] = temps
df['precipitation_mm'] = precips

# 4. View Result
print(df.head())

# Save enriched data
df.to_csv('gbif_with_weather.csv', index=False)

Fetching weather data... this may take a moment.
